In [1]:
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;sns.set_style('whitegrid',{'grid.color':'0.95'})
import matplotlib.dates as mdates
import statsmodels.formula.api as smf
import statsmodels.api as sm
import datetime
import numpy.linalg as la

from matplotlib.dates import DateFormatter
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import BetweenOLS
from scipy import stats
from stargazer.stargazer import Stargazer
from linearmodels.panel import compare
from collections import OrderedDict 

In [2]:
url='http://www.principlesofeconometrics.com/poe5/data/csv/'
nls=pd.read_csv(url+'nls_panel.csv')
vars=nls[['id','year','lwage','educ','exper','tenure','south','black']]
print(vars.head(10),'\n')

   id  year     lwage  educ      exper    tenure  south  black
0   1    82  1.808289    12   7.666667  7.666667      0      1
1   1    83  1.863417    12   8.583333  8.583333      0      1
2   1    85  1.789367    12  10.179490  1.833333      0      1
3   1    87  1.846530    12  12.179490  3.750000      0      1
4   1    88  1.856449    12  13.621790  5.250000      0      1
5   2    82  1.280933    17   7.576923  2.416667      0      0
6   2    83  1.515855    17   8.384615  3.416667      0      0
7   2    85  1.930170    17  10.384610  5.416667      0      0
8   2    87  1.919034    17  12.038460  0.333333      0      0
9   2    88  2.200974    17  13.211540  1.750000      0      0 



In [3]:
nls['year']=pd.to_datetime(nls['year'].astype(str),format='%y')
nls['year']=nls['year'].dt.year
nls=nls.set_index(['id','year'])
print(nls[['lwage','educ','exper','tenure','south','black']].head(10))

            lwage  educ      exper    tenure  south  black
id year                                                   
1  1982  1.808289    12   7.666667  7.666667      0      1
   1983  1.863417    12   8.583333  8.583333      0      1
   1985  1.789367    12  10.179490  1.833333      0      1
   1987  1.846530    12  12.179490  3.750000      0      1
   1988  1.856449    12  13.621790  5.250000      0      1
2  1982  1.280933    17   7.576923  2.416667      0      0
   1983  1.515855    17   8.384615  3.416667      0      0
   1985  1.930170    17  10.384610  5.416667      0      0
   1987  1.919034    17  12.038460  0.333333      0      0
   1988  2.200974    17  13.211540  1.750000      0      0


In [4]:
print(nls[['lwage','educ','exper','tenure','south','black']].describe(percentiles=[]
                                                                     ).transpose())

         count       mean       std       min        50%        max
lwage   3580.0   1.918238  0.464607  0.137109   1.896619   4.254619
educ    3580.0  13.022346  2.444020  4.000000  12.000000  18.000000
exper   3580.0  12.028583  3.862796  1.057692  12.054485  27.192300
tenure  3580.0   6.947439  5.171849  0.000000   6.083333  24.750000
south   3580.0   0.424022  0.494263  0.000000   0.000000   1.000000
black   3580.0   0.282123  0.450096  0.000000   0.000000   1.000000


In [5]:
url='http://www.principlesofeconometrics.com/poe5/data/csv/'
chem=pd.read_csv(url+'chemical2.csv',parse_dates=['year'])
chem['year']=chem['year'].dt.year
print(chem.dtypes)

year             int32
firm             int64
lsales         float64
lcapital       float64
llabor         float64
sk_labor       float64
lmaterials     float64
foreign        float64
export           int64
intangibles      int64
ownership        int64
dtype: object


In [6]:
vars=chem[['firm','year','lsales','lcapital','llabor']]    
print(vars.head(6),'\n')
print(vars.tail(6))

   firm  year     lsales   lcapital    llabor
0     1  2004  10.421390   9.848661  5.308268
1     1  2005  10.879330  10.065050  5.438079
2     1  2006  11.282740  10.093900  5.493062
3     2  2004   9.355739   8.774158  4.025352
4     2  2005   9.313799   8.874867  4.025352
5     2  2006   9.574984   8.874867  4.043051 

     firm  year     lsales  lcapital    llabor
594   199  2004  10.340130  8.055158  4.787492
595   199  2005   9.758462  8.872066  4.700480
596   199  2006   9.895959  9.052516  4.787492
597   200  2004   8.836374  7.649693  4.007333
598   200  2005   9.126959  8.143517  4.007333
599   200  2006   9.879707  8.470101  4.007333


In [7]:
chem_t2=chem.copy()
chem_t2=chem_t2.sort_values(['firm','year'])
chem_t2=chem_t2[chem_t2['year']>2004][['lsales','lcapital','llabor','firm','year']] 
res=smf.ols(formula='lsales~lcapital+llabor',data=chem_t2).fit() 
print(res.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.502
No. Observations:                 400   F-statistic:                     201.9
Covariance Type:            nonrobust   Prob (F-statistic):           3.28e-61
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.8745      0.211     27.885      0.000       5.460       6.289
lcapital       0.2536      0.035      7.155      0.000       0.184       0.323
llabor         0.4264      0.058      7.391      0.000       0.313       0.540

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [8]:
chem_t2=chem_t2.set_index(['firm'])
chem_t2['lsalesd']=chem_t2.groupby('firm')['lsales'].diff()
chem_t2['lcapitald']=chem_t2.groupby('firm')['lcapital'].diff()
chem_t2['llabord']=chem_t2.groupby('firm')['llabor'].diff()
res=smf.ols(formula='lsalesd~lcapitald+llabord-1', data=chem_t2).fit() 
print(res.summary(slim=True))

                   OLS Regression Results                  
Dep. Variable:                lsalesd   F-statistic:       
Model:                            OLS   Prob (F-statistic):
No. Observations:                 200                      
Covariance Type:            nonrobust                      
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lcapitald      0.0384      0.051      0.757      0.450      -0.062       0.138
llabord        0.3097      0.075      4.104      0.000       0.161       0.459

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [9]:
nls=pd.read_csv(url+'nls_panel.csv',parse_dates=['year'],date_format='%y')
nls['year']=pd.to_datetime(nls['year'].astype(str))
nls['year']=nls['year'].dt.year
nls=nls[nls['year']>1986]
nls=nls.set_index(['id','year'])
print(nls[['lwage','educ','exper']].head(6))

            lwage  educ     exper
id year                          
1  1987  1.846530    12  12.17949
   1988  1.856449    12  13.62179
2  1987  1.919034    17  12.03846
   1988  2.200974    17  13.21154
3  1987  2.007068    12  16.41667
   1988  2.089854    12  17.82051


In [10]:
nls['lwaged']=nls.groupby('id')['lwage'].diff()
nls['educd']=nls.groupby('id')['educ'].diff()
nls['experd']=nls.groupby('id')['exper'].diff()
res=smf.ols(formula='lwaged~educd+experd-1', data=nls).fit() 
print(res.summary(slim=True))

                   OLS Regression Results                  
Dep. Variable:                 lwaged   F-statistic:       
Model:                            OLS   Prob (F-statistic):
No. Observations:                 716                      
Covariance Type:            nonrobust                      
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
educd               0          0        nan        nan           0           0
experd         0.0218      0.007      3.059      0.002       0.008       0.036

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.


/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [11]:
print(nls['educd'].describe(percentiles=[]).transpose())

count    716.0
mean       0.0
std        0.0
min        0.0
50%        0.0
max        0.0
Name: educd, dtype: float64


In [12]:
res=smf.ols(formula='lwaged~experd-1',data=nls).fit()
print(res.summary(slim=True))

                   OLS Regression Results                  
Dep. Variable:                 lwaged   F-statistic:       
Model:                            OLS   Prob (F-statistic):
No. Observations:                 716                      
Covariance Type:            nonrobust                      
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
experd         0.0218      0.007      3.059      0.002       0.008       0.036

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [13]:
l=chem_t2[['lsales','lcapital','llabor']]
for i in l:
    chem_t2[str(i)+'_d']=chem_t2.groupby(chem_t2.index)[str(i)].diff()
    chem_t2[str(i)+'_m']=chem_t2.groupby(chem_t2.index)[str(i)].mean()
    chem_t2[str(i)+'_w']=chem_t2[i]-chem_t2[str(i)+'_m']
print(chem_t2[['year','lsales','lsales_d','lsales_m','lsales_w']].head(4))

      year     lsales  lsales_d   lsales_m  lsales_w
firm                                                
1     2005  10.879330       NaN  11.081035 -0.201705
1     2006  11.282740  0.403410  11.081035  0.201705
2     2005   9.313799       NaN   9.444392 -0.130593
2     2006   9.574984  0.261185   9.444392  0.130593


In [14]:
res=smf.ols(formula='lsales_w~lcapital_w+llabor_w-1',data=chem_t2).fit() 
print(res.summary(slim=True))

                   OLS Regression Results                  
Dep. Variable:               lsales_w   F-statistic:       
Model:                            OLS   Prob (F-statistic):
No. Observations:                 400                      
Covariance Type:            nonrobust                      
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lcapital_w     0.0384      0.036      1.073      0.284      -0.032       0.109
llabor_w       0.3097      0.053      5.819      0.000       0.205       0.414

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [15]:
cf=np.sqrt((res.nobs-res.df_model)/(res.nobs-chem_t2.index.nunique()-res.df_model))
print(f'correction factor  = {cf:.4f}')
se2=cf*res.bse['lcapital_w']
se3=cf*res.bse['llabor_w']
print(f'se2                = {se2:.4f}')
print(f'se3                = {se3:.4f}')

correction factor  = 1.4178
se2                = 0.0507
se3                = 0.0755


In [16]:
chem_t2=chem_t2.set_index(['year'],append=True)
chem_t2['cons']=1
exog=chem_t2[['lcapital','llabor','cons']]
res=PanelOLS(chem_t2.lsales,exog,entity_effects=True).fit()
print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 lsales   R-squared:                        0.0806
Estimator:                   PanelOLS   R-squared (Between):              0.3484
No. Observations:                 400   R-squared (Within):               0.0806
Date:                Wed, Jun 18 2025   R-squared (Overall):              0.3422
Time:                        09:28:40   Log-likelihood                    128.22
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      8.6759
Entities:                         200   P-value                           0.0002
Avg Obs:                       2.0000   Distribution:                   F(2,198)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             8.6759
                            

In [17]:
chem_t3=chem.copy()
chem_t3=chem_t3.set_index(['firm'])
l=chem_t3[['lsales','lcapital','llabor']]
for i in l:
    chem_t3[str(i)+'_d']=chem_t3.groupby(chem_t3.index)[str(i)].diff()
    chem_t3[str(i)+'_m']=chem_t3.groupby(chem_t3.index)[str(i)].mean()
    chem_t3[str(i)+'_w']=chem_t3[i]-chem_t3[str(i)+'_m']
print(chem_t3[['lsales','lsales_d','lsales_m','lsales_w']].head(6))

         lsales  lsales_d   lsales_m  lsales_w
firm                                          
1     10.421390       NaN  10.861153 -0.439763
1     10.879330  0.457940  10.861153  0.018177
1     11.282740  0.403410  10.861153  0.421587
2      9.355739       NaN   9.414841 -0.059102
2      9.313799 -0.041940   9.414841 -0.101042
2      9.574984  0.261185   9.414841  0.160143


In [18]:
chem_t3=chem_t3.reset_index()
res=smf.ols(formula='lsales_w~lcapital_w+llabor_w-1', data=chem_t3).fit() 
print(res.summary(slim=True))
cf=np.sqrt((res.nobs-res.df_model)/(res.nobs-chem_t3['firm'].nunique()-res.df_model))
print('\n')
print(f'Corrected Standard Errors of the Estimates')
print(f'correction factor = {cf:.4f}')
se2=cf*res.bse['lcapital_w']
se3=cf*res.bse['llabor_w']
print(f'corrected se2     = {se2:.4f}')
print(f'corrected se3     = {se3:.4f}')

                   OLS Regression Results                  
Dep. Variable:               lsales_w   F-statistic:       
Model:                            OLS   Prob (F-statistic):
No. Observations:                 600                      
Covariance Type:            nonrobust                      
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lcapital_w     0.0889      0.027      3.280      0.001       0.036       0.142
llabor_w       0.3522      0.041      8.521      0.000       0.271       0.433

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.


Corrected Standard Errors of the Estimates
correction factor = 1.2258
corrected se2     = 0.0332
corrected se3     = 0.0507


In [19]:
chem_t3=chem_t3.set_index(['firm','year'])
exog_vars=['lcapital','llabor']
exog=sm.add_constant(chem[exog_vars])
res=PanelOLS.from_formula('lsales~1+lcapital+llabor+EntityEffects',data=chem_t3).fit()
print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 lsales   R-squared:                        0.1252
Estimator:                   PanelOLS   R-squared (Between):              0.4272
No. Observations:                 600   R-squared (Within):               0.1252
Date:                Wed, Jun 18 2025   R-squared (Overall):              0.4132
Time:                        09:28:40   Log-likelihood                    5.8503
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      28.491
Entities:                         200   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                   F(2,398)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             28.491
                            

In [20]:
chem_DV=chem.copy()
chemD=pd.get_dummies(chem_DV.firm,dtype=float)
chem_DV=pd.concat([chem_DV,chemD],axis=1,join='inner')
print(chem_DV[['year','firm','lsales','lcapital','llabor',1,2,3,4,5,6]].head())

   year  firm     lsales   lcapital    llabor    1    2    3    4    5    6
0  2004     1  10.421390   9.848661  5.308268  1.0  0.0  0.0  0.0  0.0  0.0
1  2005     1  10.879330  10.065050  5.438079  1.0  0.0  0.0  0.0  0.0  0.0
2  2006     1  11.282740  10.093900  5.493062  1.0  0.0  0.0  0.0  0.0  0.0
3  2004     2   9.355739   8.774158  4.025352  0.0  1.0  0.0  0.0  0.0  0.0
4  2005     2   9.313799   8.874867  4.025352  0.0  1.0  0.0  0.0  0.0  0.0


In [21]:
D=chem_DV.iloc[:,11:]
res=smf.ols(formula='lsales~lcapital+llabor+D-1',data=chem_DV).fit() 
print(res.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.939
No. Observations:                 600   F-statistic:                     46.91
Covariance Type:            nonrobust   Prob (F-statistic):          2.10e-198
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lcapital       0.0889      0.033      2.676      0.008       0.024       0.154
llabor         0.3522      0.051      6.952      0.000       0.253       0.452
D[0]           8.0655      0.456     17.706      0.000       7.170       8.961
D[1]           7.2091      0.390     18.477      0.000       6.442       7.976
D[2]           6.8696      0.473     14.523      0.000       5.940       7.800
D[3]           6.4230      0.469     13.710      0.0

In [22]:
DV=res.params.iloc[2:]
print(f'The mean least squares dummy variable estimates: {np.round(DV.mean(),4)}')

The mean least squares dummy variable estimates: 7.5782


In [23]:
res=smf.ols(formula='lsales_w~lcapital_w+llabor_w-1',data=chem_t3).fit()
print(res.summary().tables[1])
sse_u=res.ssr
print(f'SSE_u: {np.round(sse_u,4)}')

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lcapital_w     0.0889      0.027      3.280      0.001       0.036       0.142
llabor_w       0.3522      0.041      8.521      0.000       0.271       0.433
SSE_u: 34.4515


In [24]:
chem_t3=chem_t3.reset_index()
res=smf.ols(formula='lsales~lcapital+llabor+1', data=chem_t3).fit()
print(res.summary().tables[1])
sse_r=res.ssr
N=chem_t3.firm.nunique()
T=chem_t3.year.nunique()
Ks=res.df_model
print(f'SSE_r       = {sse_r:.4f}')
F_stat=((sse_r-sse_u)/(N-1))/(sse_u/(N*T-N-Ks))
print(f'F-statistic =  {F_stat:.4f}')

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.8797      0.171     34.368      0.000       5.544       6.216
lcapital       0.2732      0.029      9.403      0.000       0.216       0.330
llabor         0.3815      0.047      8.176      0.000       0.290       0.473
SSE_r       = 425.6366
F-statistic =  22.7093


In [25]:
conv=smf.ols(formula='lsales~lcapital+llabor',data=chem_t3).fit(cov_type='nonrobust')
robust=smf.ols(formula='lsales~lcapital+llabor',data=chem_t3).fit(cov_type='HC1')
clus_ent=smf.ols(formula='lsales~lcapital+llabor',data=chem_t3).fit(
    cov_type='cluster',cov_kwds={'groups':chem_t3['firm']})
panel_res=Stargazer([conv,robust,clus_ent])
panel_res.custom_columns(['Conventional','Robust','Cluster-Robust'])
panel_res;

The HTML code from the previous cell is copied in this markdown cell to show the well-formatted table.

<table style="text-align:center"><tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="3"><em>Dependent variable:lsales</em></td></tr><tr><td style="text-align:left"></td><tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td></tr><tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Intercept</td><td>5.880<sup>***</sup></td><td>5.880<sup>***</sup></td><td>5.880<sup>***</sup></td></tr><tr><td style="text-align:left"></td><td>(0.171)</td><td>(0.197)</td><td>(0.328)</td></tr><tr><td style="text-align:left">lcapital</td><td>0.273<sup>***</sup></td><td>0.273<sup>***</sup></td><td>0.273<sup>***</sup></td></tr><tr><td style="text-align:left"></td><td>(0.029)</td><td>(0.035)</td><td>(0.055)</td></tr><tr><td style="text-align:left">llabor</td><td>0.382<sup>***</sup></td><td>0.382<sup>***</sup></td><td>0.382<sup>***</sup></td></tr><tr><td style="text-align:left"></td><td>(0.047)</td><td>(0.056)</td><td>(0.088)</td></tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align: left">Observations</td><td>600</td><td>600</td><td>600</td></tr><tr><td style="text-align: left">R<sup>2</sup></td><td>0.500</td><td>0.500</td><td>0.500</td></tr><tr><td style="text-align: left">Adjusted R<sup>2</sup></td><td>0.498</td><td>0.498</td><td>0.498</td></tr><tr><td style="text-align: left">Residual Std. Error</td><td>0.844 (df=597)</td><td>0.844 (df=597)</td><td>0.844 (df=597)</td></tr><tr><td style="text-align: left">F Statistic</td><td>297.997<sup>***</sup> (df=2; 597)</td><td>236.038<sup>***</sup> (df=2; 597)</td><td>84.558<sup>***</sup> (df=2; 597)</td></tr><tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align: left">Note:</td>
 <td colspan="3" style="text-align: right">
  <sup>*</sup>p&lt;0.1;
  <sup>**</sup>p&lt;0.05;
  <sup>***</sup>p&lt;0.01
 </td></tr></table> 

In [26]:
chem_t3=chem_t3.set_index(['firm','year'])
fe=PanelOLS.from_formula('lsales~1+lcapital+llabor+EntityEffects', data=chem_t3
                        ).fit(cov_type='unadjusted')
fe_rob=PanelOLS.from_formula('lsales~1+lcapital+llabor+EntityEffects', data=chem_t3
                            ).fit(cov_type='robust')
fe_clus=PanelOLS.from_formula('lsales~1+lcapital+llabor+EntityEffects', data=chem_t3
                             ).fit(cov_type='clustered',cluster_entity=True)
res=OrderedDict()
res['Conventional']=fe
res['Robust']=fe_rob
res['Cluster-Robust']=fe_clus
print(compare(res))

                         Model Comparison                         
                          Conventional       Robust Cluster-Robust
------------------------------------------------------------------
Dep. Variable                   lsales       lsales         lsales
Estimator                     PanelOLS     PanelOLS       PanelOLS
No. Observations                   600          600            600
Cov. Est.                   Unadjusted       Robust      Clustered
R-squared                       0.1252       0.1252         0.1252
R-Squared (Within)              0.1252       0.1252         0.1252
R-Squared (Between)             0.4272       0.4272         0.4272
R-Squared (Overall)             0.4132       0.4132         0.4132
F-statistic                     28.491       28.491         28.491
P-value (F-stat)                0.0000       0.0000         0.0000
=====================     ============   ==========    ===========
Intercept                       7.5782       7.5782         7.

In [27]:
res=RandomEffects.from_formula(formula='lsales~lcapital+llabor+1',data=chem_t3).fit()
print(res.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                 lsales   R-squared:                        0.2946
Estimator:              RandomEffects   R-squared (Between):              0.5033
No. Observations:                 600   R-squared (Within):               0.1058
Date:                Wed, Jun 18 2025   R-squared (Overall):              0.4849
Time:                        09:28:40   Log-likelihood                   -122.89
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      124.67
Entities:                         200   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                   F(2,597)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             124.67
                            

In [28]:
print(res.variance_decomposition)

Effects                   0.628050
Residual                  0.086561
Percent due to Effects    0.878869
Name: Variance Decomposition, dtype: float64


In [29]:
res=RandomEffects.from_formula(formula='lsales~lcapital+llabor+1',data=chem_t3
                              ).fit(cov_type='clustered',cluster_entity=True)
print(res.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      6.5429     0.2863     22.856     0.0000      5.9806      7.1051
lcapital       0.1744     0.0387     4.5022     0.0000      0.0983      0.2505
llabor         0.4227     0.0561     7.5287     0.0000      0.3124      0.5329


In [30]:
url='http://www.principlesofeconometrics.com/poe5/data/csv/'
nls=pd.read_csv(url+'nls_panel.csv')
nls['year']=pd.to_datetime(nls['year'].astype(str),format='%y')
nls['year']=nls['year'].dt.year
nls=nls.set_index(['id','year'])
formula='lwage~educ+exper+exper2+tenure+tenure2+black+south+union+EntityEffects+1'
res_fe=PanelOLS.from_formula(formula,data=nls,drop_absorbed=True).fit()
print(res_fe.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.4500     0.0401     36.124     0.0000      1.3713      1.5287
exper          0.0411     0.0066     6.2059     0.0000      0.0281      0.0541
exper2        -0.0004     0.0003    -1.4965     0.1346     -0.0009      0.0001
tenure         0.0139     0.0033     4.2433     0.0000      0.0075      0.0203
tenure2       -0.0009     0.0002    -4.3536     0.0000     -0.0013     -0.0005
south         -0.0163     0.0361    -0.4515     0.6516     -0.0872      0.0546
union          0.0637     0.0143     4.4688     0.0000      0.0357      0.0916


/var/folders/0h/jw29b3kd0ng35y2lh_0cr8bc0000gn/T/ipykernel_22290/1431450177.py:7: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ, black

  res_fe=PanelOLS.from_formula(formula,data=nls,drop_absorbed=True).fit()


In [31]:
formula='lwage~educ+exper+exper2+tenure+tenure2+black+south+union+1'
res_re=RandomEffects.from_formula(formula,data=nls).fit()
print(res_re.summary.tables[1])
print(res_re.theta.head())

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.5339     0.0799     6.6854     0.0000      0.3773      0.6905
educ           0.0733     0.0053     13.745     0.0000      0.0628      0.0837
exper          0.0436     0.0064     6.8606     0.0000      0.0312      0.0561
exper2        -0.0006     0.0003    -2.1364     0.0327     -0.0011  -4.615e-05
tenure         0.0142     0.0032     4.4697     0.0000      0.0079      0.0204
tenure2       -0.0008     0.0002    -3.8785     0.0001     -0.0011     -0.0004
black         -0.1167     0.0302    -3.8652     0.0001     -0.1759     -0.0575
south         -0.0818     0.0224    -3.6518     0.0003     -0.1258     -0.0379
union          0.0802     0.0132     6.0729     0.0000      0.0543      0.1062
       theta
id          
1   0.743593
2   0.743593


In [32]:
sig2e=res_fe.s2
res_be=BetweenOLS.from_formula(formula,data=nls).fit()
print(res_be.summary)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.3655
Estimator:                 BetweenOLS   R-squared (Between):              0.3655
No. Observations:                 716   R-squared (Within):               0.1161
Date:                Wed, Jun 18 2025   R-squared (Overall):              0.3245
Time:                        09:28:41   Log-likelihood                   -239.89
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      50.901
Entities:                         716   P-value                           0.0000
Avg Obs:                       5.0000   Distribution:                   F(8,707)
Min Obs:                       5.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             50.901
                            

In [33]:
sig2_be=res_be.resid_ss/res_be.df_resid
tbar=res_be.time_info.iloc[4]
sig2u=sig2_be-sig2e/tbar
sigu=np.sqrt(sig2u)
theta=1-np.sqrt(sig2e/(tbar*sig2u+sig2e))
print('Error Components of Variance')
print(f'sig2e   = {sig2e:.4f} variance of overall error e(it)')
print(f'sige    = {np.sqrt(sig2e):.4f} standard deviation of e(it)')
print(f'sig2_be = {sig2_be:.4f} variance from between regression')
print(f'sig2u   = {sig2u:.4f} dervied variance mu(i)')
print(f'sigu    = {sigu:.4f}  standard deviation mu(i)')
print(f'theta   = {theta:.4f} transformation parameter')

Error Components of Variance
sig2e   = 0.0381 variance of overall error e(it)
sige    = 0.1951 standard deviation of e(it)
sig2_be = 0.1159 variance from between regression
sig2u   = 0.1083 dervied variance mu(i)
sigu    = 0.3290  standard deviation mu(i)
theta   = 0.7437 transformation parameter


In [34]:
nls['cons']=1
nls['lwage_bar']=nls.groupby(['id'])['lwage'].transform('mean')
nls['lwaged']=nls['lwage']-theta*nls['lwage_bar']
cols=[]
l=nls[['educ','exper','exper2','tenure','tenure2','black','south','union','cons']]
for i in l:
    nls[str(i)+'_bar']=nls.groupby(['id'])[str(i)].transform('mean')
    nls[str(i)+'d']=nls[str(i)]-theta*nls[str(i)+'_bar']
    cols.append(str(i)+'d')
exogd=nls[cols]
res=sm.OLS(nls.lwaged,exogd).fit()
print(res.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                 lwaged   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.192
No. Observations:                3580   F-statistic:                     107.5
Covariance Type:            nonrobust   Prob (F-statistic):          3.23e-161
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
educd          0.0733      0.005     13.742      0.000       0.063       0.084
experd         0.0436      0.006      6.861      0.000       0.031       0.056
exper2d       -0.0006      0.000     -2.136      0.033      -0.001   -4.61e-05
tenured        0.0142      0.003      4.470      0.000       0.008       0.020
tenure2d      -0.0008      0.000     -3.879      0.000      -0.001      -0.000
blackd        -0.1167      0.030     -3.864      0.0

In [35]:
url='http://www.principlesofeconometrics.com/poe5/data/csv/'
chem=pd.read_csv(url+'chemical3.csv')
chem=chem.set_index(['firm','year'])
res=RandomEffects.from_formula('lsales~1+lcapital+llabor', data=chem).fit()
print(res.summary.tables[1])
tgroups=res.time_info.iloc[4]
print(f'Number of Time Periods:    {tgroups}')
ngroups=res.entity_info.iloc[4]
print(f'Number of Entities:     {ngroups}')

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      6.1718     0.1142     54.039     0.0000      5.9478      6.3957
lcapital       0.2393     0.0147     16.293     0.0000      0.2105      0.2681
llabor         0.4140     0.0220     18.784     0.0000      0.3708      0.4572
Number of Time Periods:    3.0
Number of Entities:     1000.0


In [36]:
chem=chem.reset_index()
res=smf.ols(formula='lsales~lcapital+llabor',data=chem).fit()
sse=res.ssr
chem['ehat']=res.resid
sum_ehat_id=chem.groupby(['firm'])['ehat'].sum()
sum_ehat_id_sq=sum_ehat_id**2
sum_ehat_id_sq_bar=sum_ehat_id_sq.mean()
sumtop=res.nobs*sum_ehat_id_sq_bar/tgroups
lmstat=((sumtop/sse)-1)*np.sqrt(ngroups*tgroups/(2*(tgroups-1)))
lmstatsq=lmstat**2
print('Components of the Breusch and Pagan LM Test for Random Effects')
print(f'sse       = {sse:.6}')
print(f'n         = {ngroups:.6}')
print(f't         =    {tgroups:.4}')
print(f'sumtop    = {sumtop:.6}')
print(f'lmstat    =   {lmstat:.4}')
print(f'lmstat_sq = {lmstatsq:.6}')

Components of the Breusch and Pagan LM Test for Random Effects
sse       = 2267.3
n         = 1000.0
t         =    3.0
sumtop    = 5915.34
lmstat    =   44.06
lmstat_sq = 1941.61


In [37]:
chem=chem.set_index(['firm','year']) 
res_fe=PanelOLS.from_formula('lsales~1+lcapital+llabor+EntityEffects', data=chem).fit()
b2_fe=res_fe.params.iloc[1]
se2_fe=res_fe.std_errors.iloc[1]
res_re=RandomEffects.from_formula('lsales~1+lcapital+llabor',data=chem).fit()
b2_re=res_re.params.iloc[1]
se2_re=res_re.std_errors.iloc[1]
t2_num=b2_fe-b2_re
t2_den=np.sqrt(se2_fe**2-se2_re**2)
t2=t2_num/t2_den
print(f'b2_fe  =  {b2_fe:.3}    b2_re = {b2_re:.3}  t2_num = {t2_num:.3}')
print(f'se2_fe =  {se2_fe:.3}  se2_re = {se2_re:.3} t2_num =  {t2_den:.3}')
print(f't2     = {t2:.6}')

b2_fe  =  0.116    b2_re = 0.239  t2_num = -0.123
se2_fe =  0.0195  se2_re = 0.0147 t2_num =  0.0129
t2     = -9.55494


In [38]:
def hausman(fe,re):
    b=fe.params
    B=re.params
    v_b=fe.cov
    v_B=re.cov
    df=b[np.abs(b)<1e8].size
    chi2=np.dot((b-B).T,la.inv(v_b-v_B).dot(b-B))
    pval=stats.chi2.sf(chi2,df)
    return chi2, df, pval
hausman_results=hausman(res_fe,res_re)
print(f'chi-Squared:        {str(np.round(hausman_results[0],2))}')
print(f'degrees of freedom:  {str(hausman_results[1]-1)}')
print(f'pvalue:              {np.round(hausman_results[2],6)}')

chi-Squared:        98.82
degrees of freedom:  2
pvalue:              0.0


In [39]:
exog=nls[['exper','exper2','tenure','tenure2','south','union','cons']] 
res_fe=PanelOLS(nls.lwage,exog,entity_effects=True,drop_absorbed=True).fit()
exog=nls[['educ','exper','exper2','tenure','tenure2','black','south','union','cons']] 
res_re=RandomEffects(nls.lwage,exog).fit()
def hausman(fe,re):
    b=fe.params
    b=b.drop(['cons'],axis=0)
    B=re.params
    B=B.drop(['educ','black','cons'],axis=0)
    v_b=fe.cov[['exper','exper2','tenure','tenure2','south','union']]
    v_b=v_b.drop(['cons'],axis=0)
    v_B=re.cov[['exper','exper2','tenure','tenure2','south','union']]
    v_B=v_B.drop(['educ','black','cons'],axis=0)
    df=b[np.abs(b)<1e8].size
    chi2=np.dot((b-B).T,la.inv(v_b-v_B).dot(b-B))
    pval=stats.chi2.sf(chi2,df)
    return chi2, df, pval
hausman_results=hausman(res_fe,res_re)
print(f'chi-Squared:        {np.round(hausman_results[0],4)}')
print(f'degrees of freedom:  {hausman_results[1]}')
print(f'pvalue:              {np.round(hausman_results[2],4)}')

chi-Squared:        20.745
degrees of freedom:  6
pvalue:              0.002


In [40]:
chem=chem.reset_index() 
chem['lcapitalbar']=chem.groupby(['firm'])['lcapital'].transform('mean')
chem['llaborbar']=chem.groupby(['firm'])['llabor'].transform('mean')
res=smf.ols(formula='lsales~lcapital+llabor+lcapitalbar+llaborbar',data=chem
           ).fit(cov_type='cluster',cov_kwds={'groups': chem['firm']})
print(res.summary().tables[1])
hypo='lcapitalbar=llaborbar=0'
print('Joint Hypothesis Test: lcapitalbar=llaborbar=0')
print(f'F(2,999)    = {res.f_test(hypo).fvalue:.4}')
print(f'   Prob > F =  {res.f_test(hypo).pvalue:.4}')
print(f'chisq       = {res.f_test(hypo).df_num*res.f_test(hypo).fvalue:.5}')

                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       5.4553      0.148     36.757      0.000       5.164       5.746
lcapital        0.1160      0.027      4.243      0.000       0.062       0.170
llabor          0.2689      0.046      5.868      0.000       0.179       0.359
lcapitalbar     0.2223      0.041      5.389      0.000       0.141       0.303
llaborbar       0.1095      0.062      1.760      0.078      -0.012       0.231
Joint Hypothesis Test: lcapitalbar=llaborbar=0
F(2,999)    = 28.29
   Prob > F =  1.117e-12
chisq       = 56.586


In [41]:
chem=chem.set_index(['firm','year'])
chem['cons']=1
exog=chem[['lcapital','llabor','lcapitalbar','llaborbar','cons']]
res=RandomEffects(chem.lsales,exog).fit()
print(res.summary.tables[1])
hypo='lcapitalbar=llaborbar=0'
print(res.wald_test(formula=hypo))

                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
lcapital        0.1160     0.0195     5.9367     0.0000      0.0777      0.1544
llabor          0.2689     0.0307     8.7674     0.0000      0.2087      0.3290
lcapitalbar     0.2223     0.0334     6.6582     0.0000      0.1569      0.2878
llaborbar       0.1095     0.0501     2.1853     0.0289      0.0113      0.2077
cons            5.4553     0.1372     39.768     0.0000      5.1864      5.7243
Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 97.0007
P-value: 0.0000
Distributed: chi2(2)


In [42]:
res=RandomEffects(chem.lsales, exog).fit(cov_type='clustered',cluster_entity=True)
print(res.summary.tables[1])
hypo='lcapitalbar=llaborbar=0'
print(res.wald_test(formula=hypo))

                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
lcapital        0.1160     0.0273     4.2440     0.0000      0.0624      0.1697
llabor          0.2689     0.0458     5.8696     0.0000      0.1791      0.3587
lcapitalbar     0.2223     0.0412     5.3907     0.0000      0.1415      0.3032
llaborbar       0.1095     0.0622     1.7608     0.0784     -0.0124      0.2314
cons            5.4553     0.1484     36.769     0.0000      5.1644      5.7462
Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 56.6237
P-value: 0.0000
Distributed: chi2(2)


In [43]:
nls=nls[['lwage','educ','black','south','union', 'exper','exper2','tenure','tenure2']]
cols=[]
l=nls.iloc[:,3:]
for i in l:
    nls[str(i)+'bar']=nls.groupby(['id'])[str(i)].transform('mean')
    cols.append(str(i)+'bar')
nls['cons']=1
exog=nls.iloc[:,1:]
res=RandomEffects(nls.lwage,exog).fit()    
hypo='experbar=exper2bar=tenurebar=tenure2bar=southbar=unionbar=0'
print(res.wald_test(formula=hypo))

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 20.3829
P-value: 0.0024
Distributed: chi2(6)


In [44]:
res=RandomEffects(nls.lwage,exog).fit(cov_type='clustered',cluster_entity=True)
hypo='experbar=exper2bar=tenurebar=tenure2bar=southbar=unionbar=0'
print(res.wald_test(formula=hypo))

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 17.2819
P-value: 0.0083
Distributed: chi2(6)


In [45]:
nls=nls.reset_index()
Dum=pd.get_dummies(nls.year,dtype=int)
nlsD=pd.DataFrame(Dum)
nlsD.columns=['Y1982','Y1983','Y1985','Y1987','Y1988']
nls=pd.concat([nls,nlsD],axis=1,join='inner')
nls=nls.set_index(['id','year'])
nls=nls[['lwage','educ','south','black','union','exper','exper2','tenure','tenure2',
         'experbar','exper2bar','tenurebar','tenure2bar','southbar','unionbar','Y1983',
         'Y1985','Y1987','Y1988','cons']]
exog=nls.iloc[:,1:]
res=RandomEffects(nls.lwage,exog).fit(cov_type='clustered',cluster_entity=True)
hypo='experbar=exper2bar=tenurebar=tenure2bar=southbar=unionbar=0'
print(res.wald_test(formula=hypo))

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 16.3110
P-value: 0.0122
Distributed: chi2(6)
